<a href="https://colab.research.google.com/github/w-okada/voice-changer/blob/dev/VoiceChangerDemo_Simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Voice Changer Simple (デモ版)
---

このノートはVoice ChangerをColab上で動かすデモ版です。

正式版はローカルPCのDocker上で動かすアプリケーションです。

正式版は、多くの場合より少ないタイムラグで滑らかに音声を変換できます。

詳細な使用方法はこちらの[リポジトリ](https://github.com/w-okada/voice-changer)からご確認ください。


# GPUを確認
GPUを用いたほうが高速に処理が行えます。

下記のコマンドでGPUが確認できない場合は、上のメニューから

「ランタイム」→「ランタイムの変更」→「ハードウェア アクセラレータ」

でGPUを選択してください。

In [1]:
# (1) GPUの確認
!nvidia-smi

Sun Oct 30 10:03:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 使用するモデルとコンフィグファイルの指定

使用するトレーニング済みのモデルと、トレーニングで使用したコンフィグファイルのパスを指定してください。

多くの場合はGoogle Driveに格納されているファイルを使用すると思います。その場合は、下の(2-2)のセルを実行してドライブをマウントしてください

In [ ]:
# # (2-1) 使用するモデルとコンフィグファイルの指定
# CONFIG="/content/drive/MyDrive/VoiceChanger/config.json"
# MODEL="/content/drive/MyDrive/VoiceChanger/G_326000.pth"

In [ ]:
# # (2-2) Google Driveのマウント
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


# リポジトリのクローン
リポジトリをクローンします

In [2]:
# (3) リポジトリのクローン
!git clone --depth 1 https://github.com/isletennos/MMVC_Trainer.git -b v1.3.1.3 /MMVC_Trainer
!git clone --depth 1 https://github.com/w-okada/voice-changer.git -b dev
%cd voice-changer/demo/


Cloning into '/MMVC_Trainer'...
remote: Enumerating objects: 917, done.
remote: Counting objects: 100% (917/917), done.
remote: Compressing objects: 100% (828/828), done.
remote: Total 917 (delta 3), reused 888 (delta 0), pack-reused 0
Receiving objects: 100% (917/917), 53.02 MiB | 23.35 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Note: checking out '39723b3243e0265b8802e020e4d4d90a4b8b65ee'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

Cloning into 'voice-changer'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 88 (delta 14), 

# ファイルの配置
アプリケーションの挙動を記した設定ファイルをコピーします(4-1)。(4-2)はコピーした設定ファイルを表示しています。もしかしたらうまく動かないときに役立つかもしれません。

In [3]:
# (4-1) 設定ファイルの配置
!cp ../template/setting_mmvc_colab.json ../frontend/dist/assets/setting.json


In [4]:
# (4-2) 設定ファイルの確認
!cat ../frontend/dist/assets/setting.json


{
    "app_title": "voice-changer",
    "majar_mode": "colab",
    "voice_changer_server_url": "/test",
    "sample_rate": 48000,
    "buffer_size": 1024,
    "prefix_chunk_size": 36,
    "chunk_size": 36,
    "speaker_ids": [100, 107, 101, 102, 103],
    "speaker_names": ["ずんだもん", "user", "そら", "めたん", "つむぎ"],
    "src_id": 107,
    "dst_id": 100,
    "vf_enable": true,
    "voice_changer_mode": "realtime",
    "gpu": 0,
    "available_gpus": [-1, 0, 1, 2, 3, 4],
    "avatar": {
        "enable_avatar": true,        
        "motion_capture_face": true,
        "motion_capture_upperbody": true,
        "lip_overwrite_with_voice": true,
        "avatar_url": "./assets/vrm/zundamon/zundamon.vrm",
        "backgournd_image_url": "./assets/images/bg_natural_sougen.jpg",
        "background_color": "#0000dd",
        "chroma_key": "#0000dd",
        "avatar_canvas_size": [1280, 720],
        "screen_canvas_size": [1280, 720]
    },
    "advance": {
        "avatar_draw_skip_rate": 3,
      

# モジュールのインストール

必要なモジュールをインストールします。

In [5]:
# (5) 設定ファイルの確認
!apt-get install -y espeak libsndfile1-dev
!pip install unidecode
!pip install phonemizer
!pip install retry
!pip install python-socketio
!pip install fastapi
!pip install python-multipart
!pip install uvicorn
!pip install websockets
!pip install pyOpenSSL


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1-dev is already the newest version (1.0.28-4ubuntu0.18.04.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 29 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.co

# サーバの起動

サーバを起動します。(6-1)

サーバの起動状況を確認します。(6-2) 

このセルは繰り返し実行することになるのでCtrl+Retでセルを実行してください。

アクセスできるようになるまで、１～２分かかるようです。コーヒーでも飲みに行きましょう。

下記のようなテキストが表示されたら起動完了です。

**`Application startup complete`**

という文字列で完了です。`uvicorn.error`という不穏なメッセージも見られますが、これはプロセスの名前で大して重要な意味を持ちませんので無視してください。（どうやったら消せるのだろうか。）


```
INFO:     Will watch for changes in these directories: ['/content/voice-changer/demo']
INFO:uvicorn.error:Will watch for changes in these directories: ['/content/voice-changer/demo']
INFO:     Uvicorn running on http://0.0.0.0:8088 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://0.0.0.0:8088 (Press CTRL+C to quit)
INFO:     Started reloader process [617] using StatReload
INFO:uvicorn.error:Started reloader process [617] using StatReload
INFO:uvicorn:INITIALIZE MODEL
GPU_NUM: 1
INFO:root:Loaded checkpoint '/content/drive/MyDrive/VoiceChanger/G_326000.pth' (iteration 1136)
DEBUG:asyncio:Using selector: EpollSelector
INFO:     INITIALIZE MODEL
INFO:uvicorn:INITIALIZE MODEL
GPU_NUM: 1
INFO:root:Loaded checkpoint '/content/drive/MyDrive/VoiceChanger/G_326000.pth' (iteration 1136)
INFO:     Started server process [637]
INFO:uvicorn.error:Started server process [637]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
```



In [34]:
# (6-1) サーバの起動
import random
PORT = 10000 + random.randint(1, 9999)
LOG_FILE = f"LOG_FILE_{PORT}"

get_ipython().system_raw(f'python3 MMVCServerSIO.py -p {PORT} --colab True >{LOG_FILE} 2>&1 &')
print(f"PORT:{PORT}, LOG_FILE:{LOG_FILE}")

PORT:17552, LOG_FILE:LOG_FILE_17552


In [44]:
# (6-2) サーバの起動確認 (Ctrl+Retで実行)
!tail -10 {LOG_FILE}

    Phase name:__main__
    PHASE3:__main__
    PHASE1:__main__
Start MMVC SocketIO Server
    CONFIG:None, MODEL:None
DEBUG:asyncio:Using selector: EpollSelector


# プロキシを起動
ウェブサーバへのアクセスをするためのプロキシを起動します。

表示されたURLをクリックして開くと別タブでアプリが開きます。

Colabなので、ロードにある程度時間がかかります(30秒くらい)。

In [42]:
# (7) プロキシを起動
from google.colab.output import eval_js
proxy = eval_js( "google.colab.kernel.proxyPort(" + str(PORT) + ")" )
print(f"{proxy}front/")

https://0m68en94011q-496ff2e9c6d22116-17552-colab.googleusercontent.com/front/
